In [1]:
import os
import sys
#path environment
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("please declare environment variable 'SUMO_HOME'")
import traci

import xml.etree.ElementTree as ET

In [2]:
sumo_cmd = ['sumo', 
            '--duration-log.statistics', 
            '--tripinfo-output', 'my_output_file.xml', 
            '-c', 'maps/grid/grid.sumo.cfg']

In [3]:
def analyze_time_loss(file_path):
    # Parse the XML file
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Initialize variables for analysis
    total_time_loss = 0.0
    total_waiting_time = 0.0
    total_duration = 0.0
    total_reroutes = 0
    vehicle_count = 0

    # Iterate over each tripinfo element in the XML
    for trip in root.findall('tripinfo'):
        time_loss = float(trip.get('timeLoss'))
        waiting_time = float(trip.get('waitingTime'))
        duration = float(trip.get('duration'))
        reroutes = int(trip.get('rerouteNo'))
        
        total_time_loss += time_loss
        total_waiting_time += waiting_time
        total_duration += duration
        total_reroutes += reroutes
        vehicle_count += 1

    # Calculate averages
    average_time_loss = total_time_loss / vehicle_count if vehicle_count > 0 else 0
    average_waiting_time = total_waiting_time / vehicle_count if vehicle_count > 0 else 0
    average_duration = total_duration / vehicle_count if vehicle_count > 0 else 0
    average_reroutes = total_reroutes / vehicle_count if vehicle_count > 0 else 0

    # Store results in a dictionary
    results = {
        'Total Vehicles': vehicle_count,
        'Average Time Loss': average_time_loss,
        'Average Waiting Time': average_waiting_time,
        'Average Duration': average_duration,
        'Total Reroutes': total_reroutes,
        'Average Reroutes': average_reroutes
    }
    return results


In [4]:
# Start SUMO with TraCI support
traci.start(sumo_cmd)
step = 0
while step < 3600:
    traci.simulationStep()
    
    
    step += 1

traci.close()


 Retrying in 1 seconds
***Starting server on port 44239 ***
Loading net-file from 'maps/grid/grid.net.xml' ... done (36ms).
Loading done.
Simulation version 1.18.0 started with time: 0.00.
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 1ms, vehicles TOT 4698 ACT 338 BUF 1)               
Simulation ended at time: 3600.00
Reason: TraCI requested termination.
Performance: 
 Duration: 2.68s
 TraCI-Duration: 0.21s
 Real time factor: 1345.29
 UPS: 435059.043348
Vehicles: 
 Inserted: 4698 (Loaded: 4700)
 Running: 338
 Waiting: 1
Statistics (avg of 4360):
 RouteLength: 2765.43
 Speed: 10.93
 Duration: 256.60
 WaitingTime: 15.67
 TimeLoss: 53.90
 DepartDelay: 1.54



In [5]:
results = analyze_time_loss('my_output_file.xml')


In [6]:
results

{'Total Vehicles': 4360,
 'Average Time Loss': 53.903045871559506,
 'Average Waiting Time': 15.668119266055045,
 'Average Duration': 256.5967889908257,
 'Total Reroutes': 0,
 'Average Reroutes': 0.0}